## Celebritiy Rekcognition

### Goal
The goal of the below case study, is to create an application that detects celebrieties in an image and stores the name of the first celebrity a in database for future purposes.

Below are the sequence of steps that you will be doing:

#### Preparatory Phase:

1. Use CLI to create
    - EC2 instance
    - RDS instance
    - S3 bucket
2. Create appropriate role for each service used in the application
3. Create an EC2 instance with Jupyter notebook installed


#### Implementation phase:

1. Upload an image in the S3 bucket
2. Create a Lambda function that calls Amazon Rekognition service to detect the celebrities in the image.
3. Invoke the Lambda function to extract the celebrity in the image.
4. Store the file name and celebrity name in RDS database
6. Retrieve the list of all images and details from the database and display in the notebook.
 

### Implementation Phase

#### Image Upload 

In [31]:
# installing the boto3 package
import boto3
import json


In [9]:
# importing boto3 library to access AWS services in Python
s3=boto3.client('s3')
response=s3.list_buckets()

print(response)

{'ResponseMetadata': {'RequestId': '9CC8BE4EC02797A9', 'HostId': '2Z+qlpFh8ksq9wBikLQuwe52tneBNcMtcbbxSDLqL8NCFYhikQ+6Xu8pogrLdaSBk1Fn6W7VkwQ=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '2Z+qlpFh8ksq9wBikLQuwe52tneBNcMtcbbxSDLqL8NCFYhikQ+6Xu8pogrLdaSBk1Fn6W7VkwQ=', 'x-amz-request-id': '9CC8BE4EC02797A9', 'date': 'Sat, 13 Feb 2021 09:52:58 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'demodbs3', 'CreationDate': datetime.datetime(2021, 2, 10, 8, 46, 30, tzinfo=tzlocal())}, {'Name': 'divyademo123', 'CreationDate': datetime.datetime(2021, 2, 5, 10, 11, 20, tzinfo=tzlocal())}], 'Owner': {'DisplayName': 'upgraduser2163', 'ID': '4390df6aba0fdb6ce4f528c7fe01eab053ecd31f4c7ab75061d314fee43c2175'}}


In [10]:
for bucket in response['Buckets']:
    print(bucket['Name'])

demodbs3
divyademo123


In [11]:
# Specifying the attributes for file upload
file_name="image1.jpg"
bucket_name="demodbs3"
object_name="jeff_image.jpg"

In [12]:
# initiating the S3 client
s3=boto3.client('s3')
s3.upload_file(file_name,bucket_name,object_name)

In [13]:
# Retrieve the list of existing buckets
response=s3.list_objects(Bucket="demodbs3")
print(response)

{'ResponseMetadata': {'RequestId': 'B80643258CF31C18', 'HostId': '4TIOmfSC5NZjxuoz8esGDL56c+xIZmGPGdjc68BXbDXdCsZGQxpR7PcNCVkjbNCznVFkBDLvShc=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '4TIOmfSC5NZjxuoz8esGDL56c+xIZmGPGdjc68BXbDXdCsZGQxpR7PcNCVkjbNCznVFkBDLvShc=', 'x-amz-request-id': 'B80643258CF31C18', 'date': 'Sat, 13 Feb 2021 10:04:55 GMT', 'x-amz-bucket-region': 'us-east-1', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Marker': '', 'Contents': [{'Key': 'jeff_image.jpg', 'LastModified': datetime.datetime(2021, 2, 13, 10, 3, 40, tzinfo=tzlocal()), 'ETag': '"52b591b69e38b1ef52b6b1f58fd22cac"', 'Size': 8186, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'upgraduser2163', 'ID': '4390df6aba0fdb6ce4f528c7fe01eab053ecd31f4c7ab75061d314fee43c2175'}}, {'Key': 'test.json', 'LastModified': datetime.datetime(2021, 2, 10, 8, 51, 27, tzinfo=tzlocal()), 'ETag': '"3d7c39d24c376bb4606b5038f6d

In [14]:
# List the bucket names
for object in response['Contents']:
    print(object['Key'])

jeff_image.jpg
test.json


In [ ]:
# Uploading the file on S3


In [15]:
for object in response['Contents']:
    print(object['Key'],object['Size'])

jeff_image.jpg 8186
test.json 156


In [ ]:
# Retrieving the list of objects



#### Lamda function - Creation

#Creating the handler file in the folder lambda (to be performed in separate file)

In [18]:
# Installing boto3 in a target folder

!pip3 install --target ./lambda/boto3 boto3

  Using cached boto3-1.17.7-py2.py3-none-any.whl
  Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)
  Using cached botocore-1.20.7-py2.py3-none-any.whl (7.2 MB)
  Using cached s3transfer-0.3.4-py2.py3-none-any.whl (69 kB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)
  Using cached urllib3-1.26.3-py2.py3-none-any.whl (137 kB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)


In [25]:
# Zipping the files together

%cd lambda
!zip -r9 ../celebrity_detector.zip .

[Errno 2] No such file or directory: 'lambda'
/home/ec2-user/lambda
updating: boto3/ (stored 0%)
updating: boto3/six.py (deflated 76%)
updating: boto3/six-1.15.0.dist-info/ (stored 0%)
updating: boto3/six-1.15.0.dist-info/LICENSE (deflated 41%)
updating: boto3/six-1.15.0.dist-info/METADATA (deflated 55%)
updating: boto3/six-1.15.0.dist-info/WHEEL (deflated 14%)
updating: boto3/six-1.15.0.dist-info/top_level.txt (stored 0%)
updating: boto3/six-1.15.0.dist-info/RECORD (deflated 35%)
updating: boto3/six-1.15.0.dist-info/INSTALLER (stored 0%)
updating: boto3/__pycache__/ (stored 0%)
updating: boto3/__pycache__/six.cpython-37.pyc (deflated 58%)
updating: boto3/urllib3/ (stored 0%)
updating: boto3/urllib3/__init__.py (deflated 60%)
updating: boto3/urllib3/_collections.py (deflated 69%)
updating: boto3/urllib3/_version.py (stored 0%)
updating: boto3/urllib3/connection.py (deflated 69%)
updating: boto3/urllib3/connectionpool.py (deflated 73%)
updating: boto3/urllib3/exceptions.py (deflated 69%

In [27]:
%cd ..

/home/ec2-user


In [28]:
# Creating the function
!aws lambda create-function \
--function-name celebrity_detector \
--runtime python3.7 \
--zip-file fileb://celebrity_detector.zip \
--handler handler.lamda_handle \
--role arn:aws:iam::637693265321:role/csdlambdarole \
--region us-east-1

{
    "LastUpdateStatus": "Successful", 
    "FunctionName": "celebrity_detector", 
    "LastModified": "2021-02-13T10:58:54.391+0000", 
    "RevisionId": "6274d27d-3252-40ec-9f9b-95696725f26d", 
    "MemorySize": 128, 
    "State": "Active", 
    "Version": "$LATEST", 
    "Role": "arn:aws:iam::637693265321:role/csdlambdarole", 
    "Timeout": 3, 
    "Runtime": "python3.7", 
    "TracingConfig": {
        "Mode": "PassThrough"
    }, 
    "CodeSha256": "h/e/0OO144lAgxwgxDOyJFiVKt07DjY8k9+h4Z+q4mg=", 
    "Description": "", 
    "CodeSize": 8725858, 
    "FunctionArn": "arn:aws:lambda:us-east-1:637693265321:function:celebrity_detector", 
    "Handler": "handler.lamda_handle"
}


#### Lamda function - Invoke AWS lambda

In [32]:
# Invoking the function

lam=boto3.client('lambda',region_name='us-east-1')
response=lam.invoke(
FunctionName="celebrity_detector",
InvocationType="RequestResponse",
LogType='Tail',
Payload=json.dumps({"bucket":"demodbs3","object_name":"jeff_image.jpg"})
)

celebrity_name=response['Payload'].read().decode('utf8')

In [33]:
# Storing and printing the results in a variable

print(celebrity_name)

"Jeff Bezos"


#### Database Operations

In [ ]:
# Installing the connector



In [ ]:
# Importing the library



In [ ]:
# Connector object



In [ ]:
# Defining the cursor object



In [ ]:
# Creating the table



In [ ]:
# Inserting the values into columns



In [ ]:
# Read the stored values from RDS



In [ ]:
# Displaying the results

